In [1]:
import os
from huggingface_hub import HfApi
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
#Load huggingface hub dataset from https://huggingface.co/datasets/DDSC/nordic-embedding-training-data

from datasets import load_dataset

dataset_1 = load_dataset("DDSC/nordic-embedding-training-data",
                      split=None,
                      token=True)

# dataset 2 has a train and a test split, join the test and train split into one dataset
dataset_2 = load_dataset("jealk/supervised-da",
                        split=None,
                        token=True)

from datasets import concatenate_datasets

#Rename column pos in dataset_2 to positive
dataset_2 = dataset_2.rename_column("pos", "positive")



README.md:   0%|          | 0.00/489 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/51.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/38.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55124 [00:00<?, ? examples/s]

In [8]:
len(dataset_2["train"])
#Combine train and test
dataset_2 = concatenate_datasets([dataset_2["train"], dataset_2["test"]])
len(dataset_2)


148324

In [ ]:
#Use map to filter out rows where query or positive is None or empty string
dataset_2 = dataset_2.map(lambda x: {"query": x["query"], "positive": x["positive"]} if x["query"] is not None and x["positive"] is not None else None)

#Drop None values
dataset_2 = dataset_2.filter(lambda x: x is not None)

len(dataset_2)

Map:   0%|          | 0/148324 [00:00<?, ? examples/s]

Filter:   0%|          | 0/148324 [00:00<?, ? examples/s]

148324

In [ ]:
#Drop all rows where positive is an empty string
dataset_2 = dataset_2.filter(lambda x: x["positive"] != "")

len(dataset_2)

Filter:   0%|          | 0/148324 [00:00<?, ? examples/s]

140136

In [13]:
#https://huggingface.co/datasets/jealk/supervised-da
#Add a column called 'instructions'
#For all rows with dataset_name = wiki_queries_gemma, set instructions to 'Given the title of a wikipedia article, return the article's content'
#for all rows with dataset_name = news , set instructions to "Given the title of a news article, return the article's content
#for all rows with dataset_name = europarl, set instruction to "Given a paragraph in from the european parliament in Swedish, return the paragraph in Danish"
#for all rows with dataset_name = opensubtitles_da_no, set instruction to "Given a sentence in Norwegian, return the sentence in Danish"
#For all rows with dataset_name = wiki_qa, set instruction to "Given a question, return the answer"
#For all rows with dataset_name = folketinget, set instruction to "Given a statement from the Danish parliament, return the reply from another member of the parliament"
#For all rows with dataset_name = hestenet, set instructions to "Given a post on a forum, return the response from the other users"

dataset_name_to_instructions = {
    "wiki_queries_gemma": "Given the title of a wikipedia article, return the article's content",
    "news": "Given the title of a news article, return the article's content",
    "europarl": "Given a paragraph in from the european parliament in Swedish, return the paragraph in Danish",
    "opensubtitles_da_no": "Given a sentence in Norwegian, return the sentence in Danish",
    "wiki_qa": "Given a question, return the answer",
    "folketinget": "Given a statement from the Danish parliament, return the reply from another member of the parliament",
    "hestenet": "Given a post on a forum, return the response from the other users"
}

#Use map to add the instructions column
dataset_2 = dataset_2.map(lambda x: {"instructions": dataset_name_to_instructions[x["dataset_name"]]})


Map:   0%|          | 0/140136 [00:00<?, ? examples/s]

In [15]:
#Add a new column called 'language' set it to danish for all
dataset_2 = dataset_2.map(lambda x: {"language": "danish"})

Map:   0%|          | 0/140136 [00:00<?, ? examples/s]